# Exploring Queues
<i>Adapted for use with FABRIC from [Exploring Queues](https://www.cs.unc.edu/Research/geni/geniEdu/09-queues.html)</i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)

This experiment will show the relation between queue size of a throttled router and packet loss/delay. UDP traffic will be sent over a bottlenecked router to develop the target queue.

## 1. Design the Experiment
### 1.1 Reserve Resources

#### Import the Fabric API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

#### Create slice

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name="Queues")
    
    #Router
    router = slice.add_node(name="router", site="MAX")
    router.set_capacities(cores=4, ram=16, disk=50)
    router.set_image("default_ubuntu_20")
    rPort1 = router.add_component(model='NIC_Basic', name="rPort1").get_interfaces()[0] 
    rPort2 = router.add_component(model='NIC_Basic', name="rPort2").get_interfaces()[0] 
    
    #Host 1
    host1 = slice.add_node(name="Source", site="MAX")
    host1.set_capacities(cores=4, ram=16, disk=50)
    host1.set_image("default_ubuntu_20")
    h1Port = host1.add_component(model='NIC_Basic', name="h1Port").get_interfaces()[0] 
    
    #Host 2
    host2 = slice.add_node(name="Destination", site="MAX")
    host2.set_capacities(cores=4, ram=16, disk=50)
    host2.set_image("default_ubuntu_20")
    h2Port = host2.add_component(model='NIC_Basic', name="h2Port").get_interfaces()[0] 
    
    lan1 = slice.add_l2network(name="Lan1", interfaces=[rPort1, h1Port])
    lan2 = slice.add_l2network(name="Lan2", interfaces=[rPort2, h2Port])
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    host1 = slice.get_node(name="Source") 
    host2 = slice.get_node(name="Destination")
    router = slice.get_node(name="router")
    
    subnet1 = IPv4Network("10.1.1.0/24")
    subnet2 = IPv4Network("11.1.1.0/24")
    
    host1_iface = host1.get_interface(network_name="Lan1")
    host1_iface.ip_addr_add(addr="10.1.1.1", subnet=subnet1)
    
    router_iface = router.get_interface(network_name="Lan1")
    router_iface.ip_addr_add(addr="10.1.1.2", subnet=subnet1)
    
    router_iface2 = router.get_interface(network_name="Lan2")
    router_iface2.ip_addr_add(addr="11.1.1.2", subnet=subnet2)
    
    host2_iface2 = host2.get_interface(network_name="Lan2")
    host2_iface2.ip_addr_add(addr="11.1.1.1", subnet=subnet2) 
    
    host1.execute("sudo ip route add 11.1.1.0/24 via 10.1.1.2")
    router.execute("sudo sysctl -w net.ipv4.ip_forward=1")
    host2.execute("sudo ip route add 10.1.1.0/24 via 11.1.1.2")
except Exception as e:
    print(f"Exception: {e}")


In [ ]:
for node in slice.get_nodes():
    node.execute("sudo apt update;sudo apt -y install iperf")

## 2. Experiment
### 2.1 Measuring Loss and Delay
The following steps will show how to measure packet loss and RTT.
1. Open two terminals on the `Source` nodes, one on the `Destination` node and one on the `router` node.

2. Run the following command on the `Destination` node to receive UDP messages:
<br>`iperf -s -u`

3. On one of the `Source` terminal, type but do not run:
<br>`ping 11.1.1.1`

4. On another `Source` terminal, type but do not run:
<br>`iperf -c 11.1.1.1 -u -b 20mb -t 10 &` 
<br> This command will send the Destination node UDP traffic at a bandwidth of 20mb/s for 10 seconds.

5. On the source terminals start the Iperf traffic then quickly start the ping command. When the Iperf completes, end the ping with `Ctrl+C`.

6. Your Iperf results should be similar to:
<br>` [  3]  0.0-11.4 sec  23.8 MBytes  17.5 Mbits/sec   0.086 ms    0/17007 (0%) `
<br> The `0/17007 (0%)` shows a 0% packet loss. The ping RTT should be consistent over the duration.

7. This measurement of loss will be used throughout the rest of the experiment.

### 2.2 Throttling the Router
In step 2.1 it is seen that there is no packet loss and little delay, this is due to the nodes being able to handle more traffic than we were generating (20mbps). Seeing queueing behavior requires the connection to be throttled.
1. Go to the `router` terminal and run:
<br>`sudo tc qdisc add dev ens7 root tbf rate 18mbit limit 1000mb burst 10kb`
<br>This command will change the outgoing interface to send at a rate of 18mb/s.

2. Rerun the experiment from part `2.1`, on `Source` terminals simultaneously run:
<br>`ping 11.1.1.1`
<br>`iperf -c 11.1.1.1 -u -b 20mb -t 10 &` 
<br> There will be no packet loss on the iperf report, however, there will be an increase in the delay on the ping command during the iperf execution.

### 2.3 Finite Queue

1. Adjust the queue size to 3mb on the `router` node with:
<br>`sudo tc qdisc replace dev ens7 root tbf rate 18mbit limit 3mb burst 10kb`

2. Rerun the experiment from part `2.1`, on `Source` terminals simultaneously run:
<br>`ping 11.1.1.1`
<br>`iperf -c 11.1.1.1 -u -b 20mb -t 10 &` 
<br> There may be some packet loss reported now, in this case the queue was too small for all the traffic.

### 2.4 Smaller Queue

1. Adjust the queue size to 2mb on the `router` node with:
<br>`sudo tc qdisc replace dev ens7 root tbf rate 18mbit limit 2mb burst 10kb`

2. Rerun the experiment from part `2.1`, on `Source` terminals simultaneously run:
<br>`ping 11.1.1.1`
<br>`iperf -c 11.1.1.1 -u -b 20mb -t 10 &` 
<br> Modest packet loss should be observed now. On the terminal running ping it can be observed that delay will increase until becoming constant. This occurs when a finite queue fills up, giving it a constant size.

### 2.5 Even Smaller Queue

1. Adjust the queue size to 100kb (.1 mb) on the `router` node with:
<br>`sudo tc qdisc replace dev ens7 root tbf rate 18mbit limit 100kb burst 10kb`

2. Rerun the experiment from part `2.1`, on `Source` terminals simultaneously run:
<br>`ping 11.1.1.1`
<br>`iperf -c 11.1.1.1 -u -b 20mb -t 10 &` 
<br> Packet loss will be larger while delay plateaus faster.

## 3. Cleanup Resources
### 3.1 Delete Slice

In [ ]:
try:
    slice = fablib.get_slice("Queues")
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")